# Проект "Дашборд для Яндекс.Дзен"

Содержание проекта:
* [Описание проекта -- ТЗ](#task_definition)
* [Получение данных](#data_acquisition)
* [Предобработка данных](#data_preprocessing)
* [Экспорт в csv](#csv_creation)
* [Дашборд и презентация](#results)

# Описание проекта <a class="anchor" id="task_definition"></a>

В данном проекте мы будем помогать менеджерам Яндекс.Дзен отвечать на вопросы про карточки. Нас будет интересовать пользовательское взаимодействие с карточками статей.

Пообщавшись с менеджерами и администраторами баз данных, мы составили **краткое ТЗ**.

- Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
- Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
- Кто будет основным пользователем дашборда: менеджеры по анализу контента;
- Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек;
- По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа;
- Характер данных:
    - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников — относительные величины (% событий);
    - Соответствия тем источников темам карточек - абсолютные величины;
- Важность: все графики имеют равную важность;
- Источники данных для дашборда: дата-инженеры обещали подготовить для вас агрегирующую таблицу dash_visits. Вот её структура:
    - `record_id` — первичный ключ,
    - `item_topic` — тема карточки,
    - `source_topic` — тема источника,
    - `age_segment` — возрастной сегмент,
    - `dt` — дата и время,
    - `visits` — количество событий.
- Таблица хранится в специально подготовленной для вас базе данных `zen`;
- Частота обновления данных: один раз в сутки, в полночь по UTC;
- Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):

## Получение данных <a class="anchor" id="data_acquisition"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [3]:
query = ''' SELECT *
            FROM dash_visits
        '''

In [4]:
df = pd.io.sql.read_sql(query, con = engine)
df.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


## Предобработка данных <a class="anchor" id="data_preprocessing"></a>

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [6]:
print('Всего карточек в датасете:', df['record_id'].nunique())

Всего карточек в датасете: 30745


In [7]:
print('Всего событий в датасете:', df['visits'].sum())

Всего событий в датасете: 310207


In [8]:
# проверяем столбец с количественными данными
df['visits'].describe()

count    30745.000000
mean        10.089673
std         19.727601
min          1.000000
25%          1.000000
50%          3.000000
75%         10.000000
max        371.000000
Name: visits, dtype: float64

In [9]:
# проверяем содержание категориальных данных
print(df['item_topic'].unique())
print(df['source_topic'].unique())
print(df['age_segment'].unique())

['Деньги' 'Дети' 'Женская психология' 'Женщины' 'Здоровье' 'Знаменитости'
 'Интересные факты' 'Искусство' 'История' 'Красота' 'Культура' 'Наука'
 'Общество' 'Отношения' 'Подборки' 'Полезные советы' 'Психология'
 'Путешествия' 'Рассказы' 'Россия' 'Семья' 'Скандалы' 'Туризм' 'Шоу'
 'Юмор']
['Авто' 'Деньги' 'Дети' 'Еда' 'Здоровье' 'Знаменитости' 'Интерьеры'
 'Искусство' 'История' 'Кино' 'Музыка' 'Одежда' 'Полезные советы'
 'Политика' 'Психология' 'Путешествия' 'Ремонт' 'Россия' 'Сад и дача'
 'Сделай сам' 'Семейные отношения' 'Семья' 'Спорт' 'Строительство'
 'Технологии' 'Финансы']
['18-25' '26-30' '31-35' '36-40' '41-45' '45+']


In [14]:
print('Данные представлены с', df['dt'].dt.strftime("%Y-%m-%d %H:%M").min(), 'по', df['dt'].dt.strftime("%Y-%m-%d %H:%M").max())

Данные представлены с 2019-09-24 18:28 по 2019-09-24 19:00


In [ ]:
print('Количество явных дубликатов:', df.duplicated().sum())

**Вывод по предобработке**: Мы видим, что таблица имеет рабочий вид: названия корректные, пропусков нет, формат столбцов соответствуют содержанию, дубликатов и ошибок в категориях данных также нет. Можно перейти к выгрузке файла и продолжению работы.

## Экспорт в csv <a class="anchor" id="csv_creation"></a>

In [ ]:
df.to_csv('dash_visits.csv', index = False, sep = '\t')

## Дашборд и презентация <a class="anchor" id="results"></a>

Результаты работы:

* [Дашборд](https://public.tableau.com/views/tableau_project_frolov_da56/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)
* [Презентация](https://drive.google.com/file/d/1d6CSzdwC12Kq8s037I50fp1YZW_IaMoK/view?usp=share_link)